In [56]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt 
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score 
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split, cross_val_score 
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from lightgbm import LGBMClassifier
from sklearn.experimental import enable_hist_gradient_boosting  
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from catboost import CatBoostClassifier, Pool

In [104]:
# Importing the cleaning dataset
df_imported = pd.read_csv(r"\final_clean_dataset.csv")

##### 1) Data preparation

In [105]:
data = ['duration_ms', 'popularity', 'danceability',
       'energy', 'speechiness', 'instrumentalness', 'liveness', 'valence',
       'time_signature', 'n_beats', 'fade_out_duration',
       'primary_artist_popularity', 'average_artists_popularity',
       'sum_of_followers', 'explicit']

df = df_imported[data].copy()

In [106]:
genres = df_imported['genre'].unique()
mapping_genres = {}

i= 0
# mapping genre
for element in genres:
    mapping_genres[element] = i
    i+=1

df_imported['genre'] = df_imported['genre'].map(mapping_genres)

In [107]:
X = df[data].values
y = np.array(df_imported['genre'])

In [108]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

In [109]:
mapping_genres

{'minimal-techno': 0,
 'hip-hop': 1,
 'reggae': 2,
 'honky-tonk': 3,
 'house': 4,
 'trance': 5,
 'idm': 6,
 'indian': 7,
 'indie-pop': 8,
 'indie': 9,
 'industrial': 10,
 'iranian': 11,
 'j-dance': 12,
 'j-idol': 13,
 'j-pop': 14,
 'jazz': 15,
 'j-rock': 16,
 'k-pop': 17,
 'alternative': 18,
 'alt-rock': 19,
 'ambient': 20,
 'anime': 21,
 'black-metal': 22,
 'bluegrass': 23,
 'blues': 24,
 'brazil': 25,
 'breakbeat': 26,
 'sleep': 27,
 'songwriter': 28,
 'soul': 29,
 'chill': 30,
 'spanish': 31,
 'study': 32,
 'swedish': 33,
 'synth-pop': 34,
 'tango': 35,
 'techno': 36,
 'trip-hop': 37,
 'turkish': 38,
 'world-music': 39,
 'edm': 40,
 'electro': 41,
 'latin': 42,
 'electronic': 43,
 'emo': 44,
 'folk': 45,
 'forro': 46,
 'french': 47,
 'funk': 48,
 'garage': 49,
 'german': 50,
 'gospel': 51,
 'goth': 52,
 'grindcore': 53,
 'groove': 54,
 'grunge': 55,
 'guitar': 56,
 'acoustic': 57,
 'afrobeat': 58,
 'happy': 59,
 'hard-rock': 60,
 'hardcore': 61,
 'hardstyle': 62,
 'heavy-metal': 63,

### Gradient Boosting 

In [ ]:
clf = GradientBoostingClassifier(
    max_depth=5,                # Reducing depth to control model complexity
    learning_rate=0.04,         # Lower learning rate for smoother convergence
    n_estimators=1000,          # Number of boosting rounds or trees to build
    subsample=1.0,              # Fraction of samples used for fitting the individual base learners
    min_samples_split=2,        # Minimum number of samples required to split an internal node
    min_samples_leaf=1,         # Minimum number of samples required to be at a leaf node
    min_weight_fraction_leaf=0.0,# Minimum weighted fraction of the sum total of weights required to be at a leaf node
    max_features=None,          # Number of features to consider when looking for the best split
    random_state=42,
    verbose=0                   # Controls the verbosity when fitting and predicting
)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

#print('Accuracy %s' % accuracy_score(y_test, y_pred))
#print('F1-score %s' % f1_score(y_test, y_pred, average=None))
print(classification_report(y_test, y_pred))

y_pred_train = clf.predict(X_train)
accuracy_train = accuracy_score(y_train, y_pred_train)

print('Train accuracy:', accuracy_train)

#### Too computationally expensive

### HGBC 

In [121]:
clf = HistGradientBoostingClassifier(
    loss='log_loss',                # Use log loss for binary or multiclass classification
    max_depth=5,                    # Equivalent to max_depth in XGBoost
    learning_rate=0.05,             # Same as XGBoost
    max_iter=1000,                   # Equivalent to n_estimators in XGBoost
    l2_regularization=0.6,            # Equivalent to reg_lambda in XGBoost
    min_samples_leaf=4,             # Must be an integer, similar to min_child_weight in XGBoost
    verbose=1,                      # Optional: for progress output
    early_stopping=True,            # Enable early stopping
    n_iter_no_change=7,            # Number of iterations with no improvement to stop training
    validation_fraction=0.1,        # Fraction of data to set aside as validation set for early stopping
    random_state=42                 # Ensure reproducibility
)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
y_pred_train = clf.predict(X_train)
accuracy_train = accuracy_score(y_train, y_pred_train)
print('Train accuracy:', accuracy_train)

print(classification_report(y_test, y_pred))

## Good results (#2)

Binning 0.007 GB of training data: 0.129 s
Binning 0.001 GB of validation data: 0.001 s
Fitting gradient boosted rounds:
[1/1000] 113 trees, 3247 leaves (28 on avg), max depth = 5, train loss: 4.03233, val loss: 4.09515, in 0.755s
[2/1000] 113 trees, 3223 leaves (28 on avg), max depth = 5, train loss: 3.79325, val loss: 3.88355, in 0.755s
[3/1000] 113 trees, 3215 leaves (28 on avg), max depth = 5, train loss: 3.61372, val loss: 3.72698, in 0.725s
[4/1000] 113 trees, 3172 leaves (28 on avg), max depth = 5, train loss: 3.46630, val loss: 3.59658, in 0.732s
[5/1000] 113 trees, 3202 leaves (28 on avg), max depth = 5, train loss: 3.34166, val loss: 3.49090, in 0.707s
[6/1000] 113 trees, 3193 leaves (28 on avg), max depth = 5, train loss: 3.23376, val loss: 3.39842, in 0.596s
[7/1000] 113 trees, 3181 leaves (28 on avg), max depth = 5, train loss: 3.13844, val loss: 3.31686, in 0.695s
[8/1000] 113 trees, 3142 leaves (27 on avg), max depth = 5, train loss: 3.05382, val loss: 3.24567, in 0.714s

## LGBM Classifier

- Good after RandomizedSearch (accuracy 0,56; #3)

In [19]:
clf = LGBMClassifier(boosting_type='gbdt',  # Options: 'gbdt', 'goss', 'dart'
                     objective='multiclass',  # 'multiclass' for multi-class classification
                     num_class=113,  # Number of classes for multiclass classification
                     max_depth=-1,  # No limit on depth
                     num_leaves=31,  # Number of leaves in one tree
                     n_estimators=100,  # Number of boosting iterations
                     subsample_for_bin=200000,  # Number of samples for constructing bins
                     reg_alpha=0.0,  # L1 regularization term on weights
                     reg_lambda=0.0,  # L2 regularization term on weights
                     random_state=42  # Random seed for reproducibility
                   )
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002277 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2836
[LightGBM] [Info] Number of data points in the train set: 61600, number of used features: 15
[LightGBM] [Info] Start training from score -4.587471
[LightGBM] [Info] Start training from score -4.522633
[LightGBM] [Info] Start training from score -5.875126
[LightGBM] [Info] Start training from score -4.696915
[LightGBM] [Info] Start training from score -4.563829
[LightGBM] [Info] Start training from score -4.757429
[LightGBM] [Info] Start training from score -4.503387
[LightGBM] [Info] Start training from score -4.500459
[LightGBM] [Info] Start training from score -4.725798
[LightGBM] [Info] Start training from score -4.546840
[LightGBM] [Info] Start training from score -4.528630
[LightGBM] [Info] Start training from score -5.044481
[LightGBM] [Info] Start training from score -4.605170
[LightGBM

C:\Users\marco\anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


                   precision    recall  f1-score   support

         acoustic       0.00      0.00      0.00       275
         afrobeat       0.02      0.01      0.02       315
         alt-rock       0.02      0.01      0.02        85
      alternative       0.01      0.00      0.01       234
          ambient       0.02      0.03      0.03       294
            anime       0.01      0.04      0.02       193
      black-metal       0.28      0.06      0.10       288
        bluegrass       0.06      0.04      0.04       302
            blues       0.00      0.01      0.01       228
           brazil       0.00      0.00      0.00       320
        breakbeat       0.08      0.07      0.07       312
          british       0.02      0.01      0.01       166
         cantopop       0.00      0.00      0.00       261
    chicago-house       0.07      0.04      0.05       284
         children       0.00      0.00      0.00       303
            chill       0.04      0.04      0.04       

C:\Users\marco\anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\marco\anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [21]:
# Define the parameter grid
param_dist = {
    'boosting_type': ['gbdt', 'goss', 'dart'],
    'num_leaves': np.arange(20, 150, 10),
    'max_depth': np.arange(-1, 20, 1),
    'n_estimators': np.arange(50, 200, 10),
    'learning_rate': np.linspace(0.01, 0.3, 30),
    'subsample_for_bin': [200000, 300000, 400000],
    'reg_alpha': np.linspace(0, 1, 10),
    'reg_lambda': np.linspace(0, 1, 10)
}

# Initialize the model
clf = LGBMClassifier(objective='multiclass',  # 'multiclass' for multi-class classification
                     num_class=113,  # Number of classes for multiclass classification
                     random_state=42  # Random seed for reproducibility
                   )

# Set up the random search with cross-validation
random_search = RandomizedSearchCV(estimator=clf, 
                                   param_distributions=param_dist, 
                                   n_iter=50,  # Number of parameter settings sampled
                                   scoring='accuracy',  # Evaluation metric
                                   cv=3,  # Number of cross-validation folds
                                   verbose=1,  # Verbosity
                                   random_state=42,  # Random seed for reproducibility
                                   n_jobs=-1  # Use all available cores
                                  )

# Fit the random search model
random_search.fit(X_train, y_train)

# Predict using the best estimator
best_clf = random_search.best_estimator_
y_pred = best_clf.predict(X_test)

# Output the best parameters
print("Best parameters found: ", random_search.best_params_)

Fitting 3 folds for each of 50 candidates, totalling 150 fits
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002032 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2836
[LightGBM] [Info] Number of data points in the train set: 61600, number of used features: 15
[LightGBM] [Info] Start training from score -4.587471
[LightGBM] [Info] Start training from score -4.522633
[LightGBM] [Info] Start training from score -5.875126
[LightGBM] [Info] Start training from score -4.696915
[LightGBM] [Info] Start training from score -4.563829
[LightGBM] [Info] Start training from score -4.757429
[LightGBM] [Info] Start training from score -4.503387
[LightGBM] [Info] Start training from score -4.500459
[LightGBM] [Info] Start training from score -4.725798
[LightGBM] [Info] Start training from score -4.546840
[LightGBM] [Info] Start training from score -4.528630
[LightGBM] [Info] Start training from score -5.044481
[

In [10]:
params = {'subsample_for_bin': 2000, 'reg_lambda': 1.0, 
          'reg_alpha': 0.4444444444444444, 'num_leaves': 20, 'n_estimators': 150, 
          'max_depth': 7, 'learning_rate': 0.3, 'boosting_type': 'dart'}


clf = LGBMClassifier(** params, 
                     objective='multiclass',  
                     num_class=113,
                     random_state=42  
                   )
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))

[LightGBM] [Warning] Using too small ``bin_construct_sample_cnt`` may encounter unexpected errors and poor accuracy.
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002124 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2808
[LightGBM] [Info] Number of data points in the train set: 70400, number of used features: 15
[LightGBM] [Info] Start training from score -4.889072
[LightGBM] [Info] Start training from score -4.516858
[LightGBM] [Info] Start training from score -4.800646
[LightGBM] [Info] Start training from score -4.498816
[LightGBM] [Info] Start training from score -4.484865
[LightGBM] [Info] Start training from score -4.571647
[LightGBM] [Info] Start training from score -4.510377
[LightGBM] [Info] Start training from score -4.543210
[LightGBM] [Info] Start training from score -4.672744
[LightGBM] [Info] Start training from score -5.681310
[LightGBM] [Info] Start training from score -4.484865


In [11]:
y_pred_train = clf.predict(X_train)
accuracy_train = accuracy_score(y_train, y_pred_train)

print('Train accuracy:', accuracy_train)

Train accuracy: 0.9288352272727273


The problem is the overfitting: 
- Train accuracy: 0.9462662337662338
- Test accuracy: 0.56

This result comes from a LGBMC (faster but simpler than others)

## XGBM

#### Parameters

- general parameters, 
- booster parameters, 
- task parameters.

In [71]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1
                                                    , random_state=4, stratify=y_train)

In [110]:
X_val, X_test_new, y_val, y_test_new = train_test_split(X_test, y_test, test_size=0.2
                                                    , random_state=4, stratify=y_test)

In [ ]:
clf = XGBClassifier(objective='multi:softmax', 
                    num_class=113,
                    max_depth=7,
                    learning_rate=0.05,
                    gamma=5,
                    n_estimators = 100,
                    reg_lambda=5,
                    tree_method='exact',
                    random_state=42
                   )

clf.fit(X_train, y_train)
eval_set=([(X_val, y_val)], early_stopping_rounds=10)

# [99]	validation_0-mlogloss:1.76512

In [87]:
y_pred_train = clf.predict(X_train)
accuracy_train = accuracy_score(y_train, y_pred_train)
print('Train accuracy:', accuracy_train)

y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

Train accuracy: 0.5993326118326119
              precision    recall  f1-score   support

           0       0.40      0.36      0.38       199
           1       0.62      0.65      0.64       288
           2       0.59      0.52      0.55       217
           3       0.77      0.83      0.80       294
           4       0.41      0.58      0.48       298
           5       0.45      0.40      0.42       273
           6       0.53      0.58      0.55       290
           7       0.51      0.45      0.48       281
           8       0.43      0.27      0.33       247
           9       0.74      0.36      0.48        90
          10       0.51      0.50      0.50       298
          11       0.82      0.90      0.86       205
          12       0.51      0.56      0.53       293
          13       0.53      0.62      0.57       300
          14       0.42      0.32      0.37       288
          15       0.73      0.72      0.72       216
          16       0.24      0.18      0.21   

C:\Users\marco\anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\marco\anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\marco\anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [88]:
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform

# Define the parameter grid
param_grid = {
    max_depth=5,               # Reducing depth to control model complexity
    learning_rate=0.04,        # Lower learning rate for smoother convergence
    gamma=0,                   # Minimum loss reduction required to make a further partition
    n_estimators=1000,          # Number of boosting rounds or trees to build
    booster='gbtree',          # Specify the booster type: gbtree
    reg_alpha=2,               # L1 regularization term on weights
    reg_lambda=1,              # L2 regularization term on weights
    min_child_weight=4,      # Minimum sum of instance weight (hessian) needed in a child        
}

# Initialize XGBoost classifier
clf = XGBClassifier(objective='multi:softmax', 
                    num_class=113,
                    tree_method='exact',
                    random_state=42,
                    n_jobs=-1 
                   )

# Setup RandomizedSearchCV with 5-fold cross-validation
random_search = RandomizedSearchCV(clf, param_distributions=param_grid, n_iter=50,
                                   scoring='accuracy', cv=5, verbose=1, random_state=42)

# Fit random search to the training data
random_search.fit(X_train, y_train)

# Print the best parameters and best score
print("Best Parameters found: ", random_search.best_params_)
print("Best Accuracy Score: {:.4f}".format(random_search.best_score_))

Fitting 5 folds for each of 50 candidates, totalling 250 fits
Best Parameters found:  {'gamma': 0, 'learning_rate': 0.24167343078899722, 'max_depth': 6, 'n_estimators': 185, 'reg_lambda': 10}
Best Accuracy Score: 0.5467


In [18]:
from imblearn.over_sampling import SMOTE
import numpy as np

# Assuming X_train and y_train are your training features and labels
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

# Combine original and synthetic data
X_train_augmented = np.vstack((X_train, X_resampled))
y_train_augmented = np.hstack((y_train, y_resampled))

In [72]:
clf = XGBClassifier(
    objective='multi:softmax', 
    num_class=113,
    max_depth=5,               # Reducing depth to control model complexity
    learning_rate=0.05,        # Lower learning rate for smoother convergence
    gamma=0,                   # Minimum loss reduction required to make a further partition
    n_estimators=1000,          # Number of boosting rounds or trees to build
    booster='gbtree',          # Specify the booster type: gbtree
    reg_alpha=2,               # L1 regularization term on weights
    reg_lambda=1,              # L2 regularization term on weights
    min_child_weight=4,      # Minimum sum of instance weight (hessian) needed in a child
    tree_method='hist',        # Use histogram-based algorithm for faster training
    random_state=42,
    n_jobs=-1                  # Parallelize computation using all available threads
)
clf.fit(X_train, y_train, eval_set=[(X_val, y_val)], early_stopping_rounds=10)

# [99]	validation_0-mlogloss:1.76512
#[999]	validation_0-mlogloss:1.66837
# [499]	validation_0-mlogloss:1.71034
# [999]	validation_0-mlogloss:1.61743

C:\Users\marco\anaconda\Lib\site-packages\xgboost\sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-mlogloss:4.43643
[1]	validation_0-mlogloss:4.26709
[2]	validation_0-mlogloss:4.13509
[3]	validation_0-mlogloss:4.02443
[4]	validation_0-mlogloss:3.93074
[5]	validation_0-mlogloss:3.84808
[6]	validation_0-mlogloss:3.77430
[7]	validation_0-mlogloss:3.70785
[8]	validation_0-mlogloss:3.64779
[9]	validation_0-mlogloss:3.59318
[10]	validation_0-mlogloss:3.54120
[11]	validation_0-mlogloss:3.49295
[12]	validation_0-mlogloss:3.44767
[13]	validation_0-mlogloss:3.40573
[14]	validation_0-mlogloss:3.36563
[15]	validation_0-mlogloss:3.32778
[16]	validation_0-mlogloss:3.29188
[17]	validation_0-mlogloss:3.25861
[18]	validation_0-mlogloss:3.22596
[19]	validation_0-mlogloss:3.19520
[20]	validation_0-mlogloss:3.16612
[21]	validation_0-mlogloss:3.13815
[22]	validation_0-mlogloss:3.11137
[23]	validation_0-mlogloss:3.08490
[24]	validation_0-mlogloss:3.06032
[25]	validation_0-mlogloss:3.03740
[26]	validation_0-mlogloss:3.01405
[27]	validation_0-mlogloss:2.99253
[28]	validation_0-mlogloss:2.9

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.05, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=4, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=1000, n_jobs=-1, num_class=113,
              num_parallel_tree=None, ...)

In [130]:
clf = XGBClassifier(
    objective='multi:softmax', 
    num_class=113,
    max_depth=4,               # Reducing depth to control model complexity
    learning_rate=0.076,        # Lower learning rate for smoother convergence
    gamma=0,                   # Minimum loss reduction required to make a further partition
    n_estimators=525,          # Number of boosting rounds or trees to build
    booster='gbtree',          # Specify the booster type: gbtree
    reg_alpha=2,               # L1 regularization term on weights
    reg_lambda=1,              # L2 regularization term on weights
    min_child_weight=4,      # Minimum sum of instance weight (hessian) needed in a child
    tree_method='hist',        # Use histogram-based algorithm for faster training
    random_state=42,
    n_jobs=-1                  # Parallelize computation using all available threads
)
clf.fit(X_train, y_train, eval_set=[(X_val, y_val)], early_stopping_rounds=10)

C:\Users\marco\anaconda\Lib\site-packages\xgboost\sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-mlogloss:4.34940
[1]	validation_0-mlogloss:4.16474
[2]	validation_0-mlogloss:4.02811
[3]	validation_0-mlogloss:3.91621
[4]	validation_0-mlogloss:3.82115
[5]	validation_0-mlogloss:3.73789
[6]	validation_0-mlogloss:3.66354
[7]	validation_0-mlogloss:3.59632
[8]	validation_0-mlogloss:3.53697
[9]	validation_0-mlogloss:3.48296
[10]	validation_0-mlogloss:3.43103
[11]	validation_0-mlogloss:3.38375
[12]	validation_0-mlogloss:3.34004
[13]	validation_0-mlogloss:3.29879
[14]	validation_0-mlogloss:3.25869
[15]	validation_0-mlogloss:3.22228
[16]	validation_0-mlogloss:3.18723
[17]	validation_0-mlogloss:3.15465
[18]	validation_0-mlogloss:3.12455
[19]	validation_0-mlogloss:3.09462
[20]	validation_0-mlogloss:3.06531
[21]	validation_0-mlogloss:3.03842
[22]	validation_0-mlogloss:3.01284
[23]	validation_0-mlogloss:2.98871
[24]	validation_0-mlogloss:2.96549
[25]	validation_0-mlogloss:2.94290
[26]	validation_0-mlogloss:2.92112
[27]	validation_0-mlogloss:2.90037
[28]	validation_0-mlogloss:2.8

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.076, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=4, max_leaves=None,
              min_child_weight=4, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=525, n_jobs=-1, num_class=113,
              num_parallel_tree=None, ...)

In [131]:
y_pred_train = clf.predict(X_train)
accuracy_train = accuracy_score(y_train, y_pred_train)
print('Train accuracy:', accuracy_train)
y_pred_val = clf.predict(X_val)
accuracy_val = accuracy_score(y_val, y_pred_val)
print('Val accuracy:', accuracy_val)
#y_pred = clf.predict(X_test_new)
mapped_val_pred = np.vectorize(lambda x: list(mapping_genres.keys())[list(mapping_genres.values()).index(x)])(y_pred_val)
mapped_val = np.vectorize(lambda x: list(mapping_genres.keys())[list(mapping_genres.values()).index(x)])(y_val)
report = classification_report(mapped_val, mapped_val_pred)
print(report)

Train accuracy: 0.834659090909091
Val accuracy: 0.5607481060606061
                   precision    recall  f1-score   support

         acoustic       0.38      0.37      0.37       217
         afrobeat       0.54      0.53      0.53       236
         alt-rock       0.25      0.15      0.18        62
      alternative       0.50      0.45      0.47       191
          ambient       0.56      0.67      0.61       225
            anime       0.58      0.48      0.53       174
      black-metal       0.64      0.67      0.65       233
        bluegrass       0.49      0.63      0.55       237
            blues       0.42      0.36      0.39       186
           brazil       0.39      0.36      0.38       234
        breakbeat       0.57      0.58      0.58       234
          british       0.62      0.31      0.41       135
         cantopop       0.48      0.57      0.52       210
    chicago-house       0.68      0.72      0.70       236
         children       0.80      0.82      0.8

In [133]:
y_pred_train = clf.predict(X_train)
accuracy_train = accuracy_score(y_train, y_pred_train)
print('Train accuracy:', accuracy_train)
y_pred = clf.predict(X_test_new)
mapped_values_pred = np.vectorize(lambda x: list(mapping_genres.keys())[list(mapping_genres.values()).index(x)])(y_pred)
mapped_values_test = np.vectorize(lambda x: list(mapping_genres.keys())[list(mapping_genres.values()).index(x)])(y_test_new)
report = classification_report(mapped_values_test, mapped_values_pred)
print(report)

Train accuracy: 0.834659090909091
                   precision    recall  f1-score   support

         acoustic       0.40      0.37      0.38        54
         afrobeat       0.58      0.49      0.53        59
         alt-rock       0.29      0.13      0.18        15
      alternative       0.52      0.54      0.53        48
          ambient       0.54      0.66      0.60        56
            anime       0.49      0.40      0.44        43
      black-metal       0.65      0.67      0.66        58
        bluegrass       0.51      0.66      0.58        59
            blues       0.31      0.20      0.24        46
           brazil       0.33      0.34      0.34        58
        breakbeat       0.61      0.64      0.63        59
          british       0.61      0.32      0.42        34
         cantopop       0.48      0.64      0.55        53
    chicago-house       0.68      0.76      0.72        59
         children       0.78      0.85      0.81        59
            chill    

In [134]:
# Function to parse and sort classification report by F1-score
def sort_classification_report(report):
    # Split the report by lines
    lines = report.strip().split('\n')
    
    # Parse class names and metrics
    class_names = []
    metrics = []
    for line in lines[2:-5]:  # Skip header and footer lines
        data = line.split()
        class_names.append(data[0])
        metrics.append([float(data[i]) for i in range(1, len(data))])
    
    # Sort by F1-score (index 2 in metrics)
    sorted_metrics = sorted(metrics, key=lambda x: x[2], reverse=True)
    
    # Generate the sorted report string
    sorted_report = '              precision    recall  f1-score   support\n\n'
    for i, class_name in enumerate(class_names):
        sorted_report += '{:<20} {:<10.2f} {:<10.2f} {:<10.2f} {:<10}\n'.format(
            class_name, sorted_metrics[i][0], sorted_metrics[i][1], sorted_metrics[i][2], sorted_metrics[i][3])
    
    # Add accuracy, macro avg, weighted avg lines
    sorted_report += '\n'
    for line in lines[-5:]:
        sorted_report += line + '\n'
    
    return sorted_report.strip()

# Sort the classification report by F1-score
sorted_report = sort_classification_report(report)
print(sorted_report)

precision    recall  f1-score   support

acoustic             0.92       0.98       0.95       48.0      
afrobeat             0.90       0.95       0.93       59.0      
alt-rock             0.83       0.93       0.87       41.0      
alternative          0.79       0.95       0.86       59.0      
ambient              0.78       0.90       0.83       48.0      
anime                0.78       0.85       0.81       59.0      
black-metal          0.76       0.86       0.81       58.0      
bluegrass            0.75       0.86       0.80       49.0      
blues                0.75       0.83       0.79       29.0      
brazil               0.78       0.80       0.79       59.0      
breakbeat            0.65       0.93       0.77       60.0      
british              0.78       0.74       0.76       43.0      
cantopop             0.69       0.78       0.74       32.0      
chicago-house        0.73       0.75       0.74       57.0      
children             0.74       0.71       0.73  

- Use L1 instead of L2 (= L2 is more appropriate when there's feature correlation)
- Add Boost parameter "gbtree"
- Add n_jobs = -1

####  5-clusters Classification task

In [64]:
df_5 = pd.read_csv(r"\df_classification_clusters.csv")

In [65]:
cluster_label_mapping = {
    "Low_insMid_en":0,
    "Low_insLow_en":1,
    "Low_insHigh_en":2,
    "High_insLow_en":3,
    "High_insHigh_en":4
}

# Map the numerical cluster labels to the string labels
df_5['cluster_label'] = df_5['cluster_label'].map(cluster_label_mapping)

# Display the DataFrame to see the result
df_5

,duration_ms,popularity,danceability,energy,speechiness,instrumentalness,liveness,valence,time_signature,n_beats,fade_out_duration,primary_artist_popularity,average_artists_popularity,sum_of_followers,explicit,cluster_label
0,290479.0,52.0,0.532,0.517,0.4930,0.535000,0.0854,0.2390,3.0,705.0,14574.0,51.0,51.0,218818.0,0.0,4
1,539229.0,45.0,0.750,0.340,0.0326,0.648000,0.1060,0.1450,4.0,1074.0,21947.0,47.0,47.0,231613.0,0.0,2
2,264200.0,39.0,0.801,0.639,0.1040,0.466000,0.0688,0.3200,3.0,516.0,5159.0,44.0,44.0,100828.0,0.0,4
3,387413.0,0.0,0.675,0.543,0.0639,0.719000,0.1120,0.0374,4.0,826.0,0.0,36.0,38.0,296613.0,0.0,4
4,248956.0,0.0,0.761,0.671,0.0418,0.917000,0.1150,0.1190,4.0,493.0,9321.0,45.0,45.0,245839.0,0.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87995,209655.0,23.0,0.466,0.982,0.0752,0.323000,0.3580,0.3700,4.0,585.0,9552.0,86.0,72.0,34880257.0,1.0,3
87996,205724.0,56.0,0.666,0.720,0.0269,0.000000,0.0793,0.3120,4.0,339.0,4738.0,47.0,47.0,46184.0,1.0,0
87997,195853.0,80.0,0.813,0.627,0.0486,0.000616,0.1310,0.8010,4.0,417.0,6367.0,86.0,82.0,30348451.0,1.0,1
87998,208698.0,60.0,0.653,0.779,0.0392,0.000001,0.0392,0.3870,4.0,359.0,11282.0,47.0,42.0,77107.0,0.0,3


In [71]:
data = ['duration_ms', 'popularity', 'danceability',
       'energy', 'speechiness', 'instrumentalness', 'liveness', 'valence',
       'time_signature', 'n_beats', 'fade_out_duration',
       'primary_artist_popularity', 'average_artists_popularity',
       'sum_of_followers', 'explicit']

X = df_5[data].values
y = np.array(df_5['cluster_label'])

In [72]:
# Splitting in train and test 
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.5, random_state=42)

In [39]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2
                                                    , random_state=4, stratify=y_train)

In [74]:
clf = XGBClassifier(
    objective='multi:softmax', 
    num_class=5,
    max_depth=5,  # Reduced depth to prevent overfitting
    learning_rate=0.07,  # Reduced learning rate
    gamma=0,  # Added penalty for splits
    n_estimators=500,  # Reduced number of trees
    reg_alpha=1,  # Added L1 regularization
    reg_lambda=1,  # Reduced L2 regularization
    min_child_weight=4,  # Increased to make the model more conservative
    tree_method='hist',
    random_state=42
)

clf.fit(X_train, y_train, eval_set=[(X_val, y_val)], early_stopping_rounds=5)

# [99]	validation_0-mlogloss:1.76512
#[999]	validation_0-mlogloss:1.66837
# [499]	validation_0-mlogloss:1.71034

[0]	validation_0-mlogloss:1.46508
[1]	validation_0-mlogloss:1.34288
[2]	validation_0-mlogloss:1.23813
[3]	validation_0-mlogloss:1.14549
[4]	validation_0-mlogloss:1.06391
[5]	validation_0-mlogloss:0.99116
[6]	validation_0-mlogloss:0.92587
[7]	validation_0-mlogloss:0.86637
[8]	validation_0-mlogloss:0.81270
[9]	validation_0-mlogloss:0.76304
[10]	validation_0-mlogloss:0.71789


C:\Users\marco\anaconda\Lib\site-packages\xgboost\sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[11]	validation_0-mlogloss:0.67667
[12]	validation_0-mlogloss:0.63813
[13]	validation_0-mlogloss:0.60297
[14]	validation_0-mlogloss:0.56997
[15]	validation_0-mlogloss:0.54010
[16]	validation_0-mlogloss:0.51201
[17]	validation_0-mlogloss:0.48579
[18]	validation_0-mlogloss:0.46131
[19]	validation_0-mlogloss:0.43878
[20]	validation_0-mlogloss:0.41721
[21]	validation_0-mlogloss:0.39724
[22]	validation_0-mlogloss:0.37904
[23]	validation_0-mlogloss:0.36159
[24]	validation_0-mlogloss:0.34538
[25]	validation_0-mlogloss:0.32988
[26]	validation_0-mlogloss:0.31592
[27]	validation_0-mlogloss:0.30266
[28]	validation_0-mlogloss:0.29004
[29]	validation_0-mlogloss:0.27844
[30]	validation_0-mlogloss:0.26745
[31]	validation_0-mlogloss:0.25720
[32]	validation_0-mlogloss:0.24739
[33]	validation_0-mlogloss:0.23839
[34]	validation_0-mlogloss:0.22991
[35]	validation_0-mlogloss:0.22156
[36]	validation_0-mlogloss:0.21399
[37]	validation_0-mlogloss:0.20679
[38]	validation_0-mlogloss:0.20021
[39]	validation_0-ml

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.07, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=4.5, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=500, n_jobs=None, num_class=5,
              num_parallel_tree=None, ...)

In [75]:
y_pred_train = clf.predict(X_train)
accuracy_train = accuracy_score(y_train, y_pred_train)
print('Train accuracy:', accuracy_train)

y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

Train accuracy: 1.0
              precision    recall  f1-score   support

           0       0.99      0.99      0.99     10880
           1       0.99      0.99      0.99     14794
           2       0.98      0.99      0.99      3199
           3       0.98      0.98      0.98     10092
           4       0.99      0.99      0.99      5035

    accuracy                           0.99     44000
   macro avg       0.99      0.99      0.99     44000
weighted avg       0.99      0.99      0.99     44000



## CatBoost

In [ ]:
clf = CatBoostClassifier(
    iterations=500,
    learning_rate=0.03,
    depth=6,
    #verbose=100, # Print every 100 iterations
    early_stopping_rounds=50,
    random_seed=42,
    verbose=100
)

In [ ]:
train_pool = Pool(X_train, y_train)
val_pool = Pool(X_val, y_val)

# Define the classifier
clf = CatBoostClassifier(
    iterations=500,            # Increase iterations
    learning_rate=0.1,          # Slightly higher learning rate
    depth=8,                    # Increase depth
    early_stopping_rounds=100,  # Increase early stopping rounds to give more chances for improvement
    random_seed=42,
    verbose=100,
    bagging_temperature=0.8,    # Additional parameter to increase complexity
    l2_leaf_reg=3               # Regularization parameter
)
# Train the model with validation set
clf.fit(train_pool, eval_set=val_pool, verbose=True)

0:	learn: 4.3769206	test: 4.3843632	best: 4.3843632 (0)	total: 2.67s	remaining: 22m 14s
1:	learn: 4.1959806	test: 4.2097884	best: 4.2097884 (1)	total: 5.08s	remaining: 21m 4s
2:	learn: 4.0307876	test: 4.0475697	best: 4.0475697 (2)	total: 7.57s	remaining: 20m 53s
3:	learn: 3.9095620	test: 3.9301839	best: 3.9301839 (3)	total: 9.94s	remaining: 20m 32s
4:	learn: 3.8098445	test: 3.8327956	best: 3.8327956 (4)	total: 12.4s	remaining: 20m 25s
5:	learn: 3.7111846	test: 3.7412826	best: 3.7412826 (5)	total: 14.8s	remaining: 20m 20s
6:	learn: 3.6334111	test: 3.6690844	best: 3.6690844 (6)	total: 17.6s	remaining: 20m 40s
7:	learn: 3.5571109	test: 3.5964362	best: 3.5964362 (7)	total: 20s	remaining: 20m 32s
8:	learn: 3.4852192	test: 3.5285227	best: 3.5285227 (8)	total: 22.6s	remaining: 20m 35s
9:	learn: 3.4249684	test: 3.4743702	best: 3.4743702 (9)	total: 25.2s	remaining: 20m 37s
10:	learn: 3.3672124	test: 3.4189645	best: 3.4189645 (10)	total: 27.7s	remaining: 20m 32s
11:	learn: 3.3185310	test: 3.3716

In [42]:
# Training evaluation
y_pred_train = clf.predict(X_train)
train_accuracy = accuracy_score(y_train, y_pred_train)
print('Training Accuracy: {:.4f}'.format(train_accuracy))

# Validation evaluation
y_pred_val = clf.predict(X_val)
val_accuracy = accuracy_score(y_val, y_pred_val)
print('Validation Accuracy: {:.4f}'.format(val_accuracy))

# Test evaluation
y_pred_test = clf.predict(X_test)
test_accuracy = accuracy_score(y_test, y_pred_test)
print('Test Accuracy: {:.4f}'.format(test_accuracy))

# Classification report for the test set
print('Classification Report for Test Set:')
print(classification_report(y_test, y_pred_test))

Training Accuracy: 0.9978
Validation Accuracy: 0.5540
Test Accuracy: 0.5382
Classification Report for Test Set:
              precision    recall  f1-score   support

           0       0.46      0.43      0.45       132
           1       0.62      0.68      0.65       192
           2       0.62      0.63      0.63       144
           3       0.83      0.87      0.85       196
           4       0.56      0.62      0.59       198
           5       0.50      0.49      0.50       182
           6       0.57      0.58      0.58       194
           7       0.52      0.53      0.53       187
           8       0.43      0.38      0.41       165
           9       0.77      0.45      0.57        60
          10       0.54      0.55      0.54       199
          11       0.81      0.91      0.86       137
          12       0.61      0.59      0.60       196
          13       0.60      0.68      0.64       200
          14       0.40      0.39      0.39       192
          15       0.81